MODEL 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Baca dataset dari file CSV
data = pd.read_excel('food_db.xlsx')

In [ ]:
import json

def extract_ingredients(row):
    try:
        bahan = json.loads(row)
        return [item['bahan'] for item in bahan]
    except Exception as e:
        return []

data['Extracted Ingredients'] = data['Bahan'].apply(extract_ingredients)

print(data['Extracted Ingredients'])

In [ ]:
# Mengisi nilai yang hilang dengan 0
data.fillna(0, inplace=True)

# Mengubah daftar bahan menjadi string
data['Extracted Ingredients'] = data['Extracted Ingredients'].apply(lambda x: [ingredient.replace('-', '') for ingredient in x])
data['Ingredients String'] = data['Extracted Ingredients'].apply(lambda x: ', '.join(x))

# data['Ingredients String']

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# Menggunakan MultiLabelBinarizer untuk encoding variabel "Ingredients String"
mlb = MultiLabelBinarizer()
bahan_encoded = mlb.fit_transform(data['Ingredients String'])

Epoch 1/10
982/982 [==============================] - 3s 3ms/step - loss: 7.0631
Epoch 2/10
982/982 [==============================] - 3s 3ms/step - loss: 6.7462
Epoch 3/10
982/982 [==============================] - 3s 3ms/step - loss: 6.1425
Epoch 4/10
982/982 [==============================] - 2s 2ms/step - loss: 5.4833
Epoch 5/10
982/982 [==============================] - 2s 2ms/step - loss: 4.9355
Epoch 6/10
982/982 [==============================] - 3s 3ms/step - loss: 4.4326
Epoch 7/10
982/982 [==============================] - 3s 3ms/step - loss: 4.0159
Epoch 8/10
982/982 [==============================] - 2s 2ms/step - loss: 3.6495
Epoch 9/10
982/982 [==============================] - 2s 2ms/step - loss: 3.3413
Epoch 10/10
982/982 [==============================] - 3s 3ms/step - loss: 3.0883


In [ ]:
# Menggunakan OneHotEncoder untuk encoding variabel "Bahan"
bahan = data['Bahan'].values.reshape(-1, 1)  # Reshape to (n_samples, 1)
encoder = OneHotEncoder(sparse=False)
bahan_encoded = encoder.fit_transform(bahan)

# bahan_encoded

In [ ]:
# Membaca atribut gizi sebagai fitur
features = data[['Kalori', 'Protein', 'Karbo', 'Lemak', 'Serat']].values

# Normalisasi fitur
features_norm = (features - features.mean(axis=0)) / features.std(axis=0)

# Membaca atribut rekomendasi sebagai target
target = data['Makanan'].astype('category').cat.codes

# Menggabungkan -0--- dan variabel "Bahan" yang telah di-encode
features_encoded = np.concatenate((features_norm, bahan_encoded), axis=1)

# Membagi data menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(features_norm, target, test_size=0.2, random_state=42)

# Membangun model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(data['Makanan'].unique()), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Melatih model
model.fit(features_norm, target, epochs=50, batch_size=1)



Loss:  2.640801191329956


In [ ]:
# Evaluasi model dengan data pengujian
loss = model.evaluate(X_test, y_test, verbose=0)

print("Loss: ", loss)


In [ ]:
# menyimpan model
model.save('my_model.h5')

In [ ]:
# fungsi tambahan
# function baru (di cek dulu alergi baru dimasukkan di list)
def get_top_5_recommendations(nutrition_data, allergies=[]):
    # Normalisasi data gizi pengguna
    nutrition_data_norm = (nutrition_data - features.mean(axis=0)) / features.std(axis=0)

    # Memprediksi probabilitas kelas makanan
    predictions = model.predict(np.array([nutrition_data_norm]))
    sorted_predictions = np.argsort(predictions[0])[::-1]
    # Mendapatkan indeks 5 kelas dengan probabilitas tertinggi
    # top_5_indices = np.argsort(predictions[0])[::-1][:5]
    recommended_foods = []

    for index in sorted_predictions:
        # Getting the food name
        food = data.loc[index, 'Makanan']

        # Checking if the food contains any allergies
        if any(allergy in food for allergy in allergies):
            continue  # Skip this food if it contains allergens

        # Adding the food to the recommended foods list
        recommended_foods.append(food)

        # Break the loop if we have 5 recommended foods
        if len(recommended_foods) == 5:
            break

    return recommended_foods

def calculate_bmr(weight, height, age, gender):
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    return bmr

In [ ]:
# Contoh penggunaan
weight = 80  # Ganti dengan berat badan pengguna dalam kilogram
height = 180  # Ganti dengan tinggi badan pengguna dalam sentimeter
age = 21  # Ganti dengan usia pengguna
gender = 'Male'  # Ganti dengan jenis kelamin pengguna ('Male' atau 'Female')
allergies = ['Ayam', 'Tauge']  # Ganti dengan daftar alergi pengguna, kosongkan jika tidak ada

bmr = calculate_bmr(weight, height, age, gender)
nutrition_data = [bmr]  # Gunakan BMR sebagai kalori makanan yang direkomendasikan

# Mendapatkan rekomendasi makanan
recommended_foods = get_top_5_recommendations(nutrition_data, allergies)
print("Rekomendasi makanan:")
for food in recommended_foods:
    print(food)

1/1 [==============================] - 0s 38ms/step
Makanan yang direkomendasikan:   Soto Banjar  
